<a href="https://colab.research.google.com/github/darthfelipe21/DataScientist2/blob/main/WebScrapping_Autolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import pandas as pd
from scrapy import Selector

In [ ]:
# Declarar variables
df = pd.DataFrame(columns=['Título del Producto', 'Compatibilidad'])

current_page = 1
max_pages = 15

# URL
base_url = 'https://pecas.vw.com.br/lista/pecas-vw?page={}'


In [ ]:
while current_page <= max_pages:
    try:
        # Iterar sobre paginas
        current_url = base_url.format(current_page)

        # Pagina actual (SI LA EJECUCION NO SE REALIZA ES POR EL "HEADERS" QUE DEBE DE SER CAMBIADO SEGUN INFO DE LA SIGUIENTE WEB https://httpbin.org/get)
        html = requests.get(current_url, headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}).content

        # Crear selector
        sel = Selector(text=html, type='html')

        # Extraer titulo y compatibilidad
        titulos_elemento = sel.xpath('//h3[@class="head-regular color-grey-3c product__list__item__desc nome-produto"]/text()').getall()
        compatibilidad = sel.xpath('//div[@class="subtitle head-regular color-grey-6a"]/b[@class="head-bold"]/text()').getall()

        # Verificar la longitud por pagina
        min_length = min(len(titulos_elemento), len(compatibilidad))

        # Crear DF temporal
        df_temporal = pd.DataFrame({'Título del Producto': titulos_elemento[:min_length], 'Compatibilidad': compatibilidad[:min_length]})

        # Agrupar elementos
        grouped_df = df_temporal.groupby('Título del Producto')['Compatibilidad'].apply(list).reset_index()

        # Cocatenar con DF
        df = pd.concat([df, grouped_df], ignore_index=True)

        # Contador de paginas
        current_page += 1

        time.sleep(1)

    except Exception as e:
        print(f"An error occurred: {e}")
        break

In [ ]:
# Imprimir DataFrame
df

,Título del Producto,Compatibilidad
0,\n\t\t\tAmortecedor de Suspensão Traseiro Dire...,[T-Cross]
1,\n\t\t\tKit Amortecedor Dianteiro T-Cross Orig...,"[Gol, Parati, Voyage, Saveiro]"
2,\n\t\t\tKit Cabeçote Parcial Gol/Voyage/Saveir...,[T-Cross]
3,\n\t\t\tKit Correia Dentada e Tensor Gol/Savei...,[T-Cross]
4,"\n\t\t\tKit Correia Dentada, Tensor e Polia O...","[Up, Gol, Voyage, Saveiro, T-Cross, Polo Track..."
...,...,...
115,\n\t\t\tCabeçote VW 04C103064E\n\t\t,"[Nivus, T-Cross, Up!]"
116,\n\t\t\tCabeçote VW 04C103064G\n\t\t,"[Golf, Jetta, Polo, T-Cross, Taos, Tiguan, Vir..."
117,\n\t\t\tCabeçote VW 04E103066Q\n\t\t,"[Golf, Jetta, Polo, T-Cross, Taos, Tiguan, Vir..."
118,\n\t\t\tEngrenagem de Câmbio VW 0803113413\n\t\t,"[Nivus, T-Cross, Up!]"


In [ ]:
from scrapy import Selector

html = """
<div data-v-73fc0c6c="" class="produto-preco"><div data-v-73fc0c6c=""><!----> <!----> <!----> <!----> <div data-v-54e214e3="" data-v-73fc0c6c="" class="list"><div data-v-54e214e3="" class="container-preco"><div data-v-54e214e3=""><div data-v-54e214e3="" class="preco-final head-bold color-grey-3c">
    R$ 881,79
</div></div> <!----> <div data-v-54e214e3="" class="retire__gratis hidden"><div data-v-54e214e3="" class="head-bold flex items-center gap-1" style="line-height: 1;"><svg data-v-54e214e3="" width="64" height="64" viewBox="0 0 64 64" fill="none" xmlns="http://www.w3.org/2000/svg" class="retire-icon hidden md:block" style="width: 16px; height: 16px;"><g data-v-54e214e3="" clip-path="url(#clip0_1209_208730)"><path data-v-54e214e3="" d="M11.88 5.4C12.2555 4.96094 12.7217 4.60844 13.2465 4.36676C13.7713 4.12509 14.3422 3.99996 14.92 4H49.08C49.6578 3.99996 50.2287 4.12509 50.7535 4.36676C51.2783 4.60844 51.7445 4.96094 52.12 5.4L62.556 17.576C63.4878 18.6633 64 20.048 64 21.48V22.5C64.0002 24.4799 63.3818 26.4104 62.2313 28.0217C61.0807 29.633 59.4555 30.8445 57.5827 31.487C55.7099 32.1294 53.6832 32.1707 51.7859 31.6051C49.8885 31.0394 48.2152 29.8951 47 28.332C46.1126 29.4751 44.9753 30.3999 43.6753 31.0357C42.3754 31.6715 40.9471 32.0013 39.5 32C38.0528 32.0016 36.6245 31.6719 35.3245 31.0361C34.0244 30.4003 32.8872 29.4753 32 28.332C31.1128 29.4753 29.9756 30.4003 28.6755 31.0361C27.3755 31.6719 25.9472 32.0016 24.5 32C23.0528 32.0016 21.6245 31.6719 20.3245 31.0361C19.0244 30.4003 17.8872 29.4753 17 28.332C15.7848 29.8951 14.1115 31.0394 12.2141 31.6051C10.3168 32.1707 8.29006 32.1294 6.4173 31.487C4.54453 30.8445 2.91929 29.633 1.76873 28.0217C0.618167 26.4104 -0.000218394 24.4799 5.78582e-08 22.5V21.48C5.04301e-05 20.048 0.512226 18.6633 1.444 17.576L11.884 5.396L11.88 5.4ZM19 22.5C19 23.9587 19.5795 25.3576 20.6109 26.3891C21.6424 27.4205 23.0413 28 24.5 28C25.9587 28 27.3576 27.4205 28.3891 26.3891C29.4205 25.3576 30 23.9587 30 22.5C30 21.9696 30.2107 21.4609 30.5858 21.0858C30.9609 20.7107 31.4696 20.5 32 20.5C32.5304 20.5 33.0391 20.7107 33.4142 21.0858C33.7893 21.4609 34 21.9696 34 22.5C34 23.9587 34.5795 25.3576 35.6109 26.3891C36.6424 27.4205 38.0413 28 39.5 28C40.9587 28 42.3576 27.4205 43.3891 26.3891C44.4205 25.3576 45 23.9587 45 22.5C45 21.9696 45.2107 21.4609 45.5858 21.0858C45.9609 20.7107 46.4696 20.5 47 20.5C47.5304 20.5 48.0391 20.7107 48.4142 21.0858C48.7893 21.4609 49 21.9696 49 22.5C49 23.9587 49.5795 25.3576 50.6109 26.3891C51.6424 27.4205 53.0413 28 54.5 28C55.9587 28 57.3576 27.4205 58.3891 26.3891C59.4205 25.3576 60 23.9587 60 22.5V21.48C60 21.0033 59.8298 20.5423 59.52 20.18L49.08 8H14.92L4.48 20.18C4.17019 20.5423 3.99996 21.0033 4 21.48V22.5C4 23.9587 4.57946 25.3576 5.61091 26.3891C6.64236 27.4205 8.04131 28 9.5 28C10.9587 28 12.3576 27.4205 13.3891 26.3891C14.4205 25.3576 15 23.9587 15 22.5C15 21.9696 15.2107 21.4609 15.5858 21.0858C15.9609 20.7107 16.4696 20.5 17 20.5C17.5304 20.5 18.0391 20.7107 18.4142 21.0858C18.7893 21.4609 19 21.9696 19 22.5ZM6 34C6.53043 34 7.03914 34.2107 7.41421 34.5858C7.78929 34.9609 8 35.4696 8 36V60H56V36C56 35.4696 56.2107 34.9609 56.5858 34.5858C56.9609 34.2107 57.4696 34 58 34C58.5304 34 59.0391 34.2107 59.4142 34.5858C59.7893 34.9609 60 35.4696 60 36V60H62C62.5304 60 63.0391 60.2107 63.4142 60.5858C63.7893 60.9609 64 61.4696 64 62C64 62.5304 63.7893 63.0391 63.4142 63.4142C63.0391 63.7893 62.5304 64 62 64H2C1.46957 64 0.960859 63.7893 0.585787 63.4142C0.210714 63.0391 5.78582e-08 62.5304 5.78582e-08 62C5.78582e-08 61.4696 0.210714 60.9609 0.585787 60.5858C0.960859 60.2107 1.46957 60 2 60H4V36C4 35.4696 4.21071 34.9609 4.58579 34.5858C4.96086 34.2107 5.46957 34 6 34ZM14 36C14.5304 36 15.0391 36.2107 15.4142 36.5858C15.7893 36.9609 16 37.4696 16 38V52H48V38C48 37.4696 48.2107 36.9609 48.5858 36.5858C48.9609 36.2107 49.4696 36 50 36C50.5304 36 51.0391 36.2107 51.4142 36.5858C51.7893 36.9609 52 37.4696 52 38V52C52 53.0609 51.5786 54.0783 50.8284 54.8284C50.0783 55.5786 49.0609 56 48 56H16C14.9391 56 13.9217 55.5786 13.1716 54.8284C12.4214 54.0783 12 53.0609 12 52V38C12 37.4696 12.2107 36.9609 12.5858 36.5858C12.9609 36.2107 13.4696 36 14 36Z" fill="#029640"></path></g> <defs data-v-54e214e3="" data-v-83db9b14=""><clipPath data-v-54e214e3="" data-v-83db9b14="" id="clip0_1209_208730"><rect data-v-54e214e3="" data-v-83db9b14="" width="64" height="64" fill="white"></rect></clipPath></defs></svg> <div data-v-54e214e3="">
    Retire grátis em: <span data-v-54e214e3="" class="head-bold color-grey-3c">Distac Duque de Caxias - RJ</span></div></div></div> <div data-v-54e214e3="" class="qtd__stock head-regular color-grey-6a mt-2 hidden">1 vendedor com estoque</div> <!----></div> <!----></div></div></div>
"""

sel = Selector(text=html)

elemento_div = sel.css('div.preco-final')

valor_data_v = elemento_div.css('::attr(data-v-54e214e3)').get()

print(f'Valor: {valor_data_v}')

Valor: 


In [ ]:
from scrapy import Selector


xpath_expresion = '//*[@id="layout-default"]/main/div/div/div[3]/div[2]/div[5]/div/a[1]/div[3]/div[2]/div/div/div/div/div[1]/div'

sel = Selector(text=html)

elemento_div = sel.xpath(xpath_expresion)

valor_data_v = elemento_div.xpath('@data-v-54e214e3').get()

print("Valor:", valor_data_v)

Valor: None


In [ ]:
import requests
from lxml import html

url = 'https://pecas.vw.com.br/lista/'
response = requests.get(url)
html_content = response.content

parsed_html = html.fromstring(html_content)
xpath_expression = '//div[@class="preco head-bold color-grey-3c"]'
precio_element = parsed_html.xpath(xpath_expression)

precio_element

[]